In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from multiprocessing import Pool, cpu_count

In [2]:
train = pd.read_csv('train.csv')
train = pd.concat((train,pd.read_csv('churn_comp_refresh/train_v2.csv')),axis=0, ignore_index=True).reset_index(drop=True)
test = pd.read_csv('churn_comp_refresh/sample_submission_v2.csv')


In [3]:
train

,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1
...,...,...
1963886,OHnZbu+EVaP+vN7Z+OfT5OMcp90MWFZonmM0o3pb8FY=,0
1963887,S92bDK//uI6hk3u1vuApro0qJBQOToBozZ7lL1yTC+w=,0
1963888,eUa3xo16vpAjr43Cjlb6Kjf1NTILYyJIkBayJQdXWnw=,0
1963889,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0


In [4]:
test 

,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0
...,...,...
907466,8nS1SbsI9+9vN07m8GyfmsWv/A9H8dUlbIbjbZ2a/Nk=,0
907467,KsOpJUA4P2pEaSl0afZhyl1QBaLHXTw33IYuQsnKdxE=,0
907468,NIcI2YUj9v2tC/NRtreLz1wxwLczQacJ2Ek9RJKgarI=,0
907469,b8MlVUtzUhQrIqKcqVKPENOIUo/5EiZxfExBkkfoBxY=,0


In [5]:
members = pd.read_csv('members_v3.csv')


In [6]:
transaction = pd.read_csv('transactions.csv')
transaction = pd.concat((transaction,pd.read_csv('churn_comp_refresh/transactions_v2.csv')),axis=0,ignore_index=True).reset_index(drop=True)


In [7]:
transaction

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0
...,...,...,...,...,...,...,...,...,...
22978750,zwF50wwaJI2TBKWhB42HRBJ6EQK0jgSo1Xmwb9Jq3SU=,32,180,536,536,0,20170215,20170817,0
22978751,zx/h5MzQQmsSat04wSfGpHp6N8aWLLwM1+7OV7ujmPY=,41,30,149,149,1,20170306,20170406,0
22978752,zxvgjIKjy18Fm+cIWUfYKr68z09+ILBxuMW0DnbeUZ8=,41,30,99,99,1,20170308,20170408,0
22978753,zzNhkExbpzmpjp9tXefiCUBtgNLgS+vZE7fFfTRDJVc=,38,30,149,149,0,20170318,20170417,0


In [8]:
transaction = transaction.sort_values(by=['transaction_date'], ascending=[False]).reset_index(drop=True)

In [9]:
transaction = transaction.drop_duplicates(subset=['msno'],keep='first')

In [10]:
## Discount
transaction['discount'] = transaction['plan_list_price']-transaction['actual_amount_paid']


In [11]:
## is_discount
transaction['is_discount'] = transaction.discount.apply(lambda x:1 if x>0 else 0)
## Membership Duration 
transaction['membership_duration'] = transaction.membership_expire_date-transaction.transaction_date

In [12]:
transaction

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_discount,membership_duration
0,nlwI4rVeDrFDFjkHBXuMBBXAWvhTv9miDiEGdaMDiXs=,41,30,99,99,1,20170331,20170501,0,0,0,170
1,GOKnXKY9XXaf3PaL9seD4HENmWw3o5WFUKTKL2irFrA=,39,30,149,149,1,20170331,20170505,0,0,0,174
2,TJ6nxPb8NDfeKb8HpZz2HemVvZHZdS52OJdMYFCZHv4=,41,30,99,99,1,20170331,20170430,0,0,0,99
3,9NwzFDrPGKgEuLKrM7EXsqg+VtebHhJ5WJFDOWhY4bQ=,41,30,99,99,1,20170331,20170430,0,0,0,99
4,TM/4vR23dnG79lZCqimPzRLBCq6RlG3ibOqs1Wd9IgY=,34,30,149,149,1,20170331,20170430,0,0,0,99
...,...,...,...,...,...,...,...,...,...,...,...,...
22978621,sCd4fIlc/HrfBOoH/igebLhi9YEfFalp70FD88hrvmc=,37,31,149,149,1,20150101,20150101,1,0,0,0
22978687,L799T0J/kageC/dZPcq6Vr7cFI34xza83NwAnTid5Xs=,38,30,149,149,0,20150101,20150131,0,0,0,30
22978691,fPzDykIjIjzzVJmT19dd4gYjxg4eorkOXBN3YI0ESx4=,41,30,149,149,1,20150101,20141231,1,0,0,-8870
22978727,zMWLi8IwUpcHSC+ED73h+9iYgTDhzT/DCftT/ao03mM=,41,30,149,0,1,20150101,20141231,1,149,1,-8870


In [13]:
train['is_train'] = 1
test['is_train'] = 0
combined = pd.concat([train,test],axis=0)

combined = pd.merge(combined, members,how='left',on='msno')
members = []; print('done Merge..')

gender = {'male':1, 'female':2}
combined['gender'] = combined['gender'].map(gender)

combined = pd.merge(combined, transaction, how='left',on='msno')
transaction=[]; print('transaction merge...')

train = combined[combined['is_train']==1]
test = combined[combined['is_train']==0]

train.drop('is_train',axis=1,inplace = True)
test.drop('is_train',axis=1,inplace=True)

del combined

done Merge..
transaction merge...


C:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train

In [ ]:
test

In [76]:
'''
def transform_df(df):
    df = pd.DataFrame(df)
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

def transform_df2(df):
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df
    
'''

"\ndef transform_df(df):\n    df = pd.DataFrame(df)\n    df = df.sort_values(by=['date'], ascending=[False])\n    df = df.reset_index(drop=True)\n    df = df.drop_duplicates(subset=['msno'], keep='first')\n    return df\n\ndef transform_df2(df):\n    df = df.sort_values(by=['date'], ascending=[False])\n    df = df.reset_index(drop=True)\n    df = df.drop_duplicates(subset=['msno'], keep='first')\n    return df\n    \n"

In [77]:
'''user_logs_all  = []

df_iter = pd.read_csv('user_logs.csv',low_memory=False,iterator=True,chunksize=2000000)
'''

"user_logs_all  = []\n\ndf_iter = pd.read_csv('user_logs.csv',low_memory=False,iterator=True,chunksize=2000000)\n"

In [78]:
'''
i=0
for data in df_iter:
    
    if i<2:  ## Loading 300M Records 
        
        if len(data)>0:
            print(data.shape)
            p = Pool(cpu_count())
            data = p.map(transform_df,np.array_split(data,cpu_count()))
            print('done')
            data = pd.concat(data,axis=0,ignore_index=True).reset_index(drop=True)
            
            data = transform_df2(data)
            
            print('done')

            p.close(); p.join()
            
            user_logs_all.append(data)
            
            print('final_shape: ',data.shape)
            
            data = []
    i+=1

'''
            
        

"\ni=0\nfor data in df_iter:\n    \n    if i<2:  ## Loading 300M Records \n        \n        if len(data)>0:\n            print(data.shape)\n            p = Pool(cpu_count())\n            data = p.map(transform_df,np.array_split(data,cpu_count()))\n            print('done')\n            data = pd.concat(data,axis=0,ignore_index=True).reset_index(drop=True)\n            \n            data = transform_df2(data)\n            \n            print('done')\n\n            p.close(); p.join()\n            \n            user_logs_all.append(data)\n            \n            print('final_shape: ',data.shape)\n            \n            data = []\n    i+=1\n\n"

In [ ]:
### Tried it on 1M dataset 

In [15]:
import time
size=1e6   # 1 million
user_log_chunks = pd.read_csv('user_logs.csv',chunksize=size,index_col=['msno'])
start_time = time.time()
for i,user_log_chunk in enumerate(user_log_chunks):
    #print(user_log_chunk.head())
    grouped_object=user_log_chunk.groupby(user_log_chunk.index,sort=False) # not sorting results in a minor speedup
    func = {'date':['min','max'], 
           'num_25':['sum','count','std','mean'],'num_50':['sum','std','mean'],
           'num_75':['sum','std','mean'],'num_985':['sum','std','mean'],
           'num_100':['sum','std','mean'],'num_unq':['sum','std','mean'],'total_secs':['sum','std','mean']}
    if(i==0):  # for the fiest run create the result 
        result=grouped_object.agg(func)
        print(len(result))
        print("Round ",i,"took %s seconds" % (time.time() - start_time))
    else:
        result=result.append(grouped_object.agg(func))
        print(len(result))
        print("Round ",i,"took %s seconds" % (time.time() - start_time))
        break

202510
Round  0 took 4.382216930389404 seconds
407351
Round  1 took 8.922666072845459 seconds


In [30]:
result = result.reset_index()

In [39]:
result

date           num_25        \
                                                   min       max    sum count   
msno                                                                            
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  20150326  20150716     75    13   
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150105  20170111    315    78   
PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=  20151201  20170125     18    18   
KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=  20150803  20170201     89    24   
oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=  20150205  20160727    167    60   
...                                                ...       ...    ...   ...   
yyPt5tJrg0A+u3ELliQHaxWCKHKuxHTJkBHkLvaGRuk=  20150406  20150406      3     1   
5EdEfCx2H/HCMLiFDgTyViSzMPLiPJrA6EFW8mqbGn8=  20161127  20161127      0     1   
+KvFWdNadmk4EsYLM1YJpMmfpZ6dWW+F6Gch4+AyI7I=  20150210  20150210      2     1   
xar5CEm9tD/GwgRa9sjN0Fe/SeH7njj5XQHjBi+MRPE=  20150328  20150328      0     1   
ZDBfOAFk7mSNPn5Ui39z9U3NY87u2kCbhmlPRZzJPBw=  20150106  20150106      0     1   

                                                                 num_50  \
                                                   std      mean    sum   
msno                                                                      
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  9.065997  5.769231     11   
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  4.590907  4.038462    119   
PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=  1.137593  1.000000      9   
KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=  5.187604  3.708333     28   
oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=  3.252596  2.783333     90   
...                                                ...       ...    ...   
yyPt5tJrg0A+u3ELliQHaxWCKHKuxHTJkBHkLvaGRuk=       NaN  3.000000      0   
5EdEfCx2H/HCMLiFDgTyViSzMPLiPJrA6EFW8mqbGn8=       NaN  0.000000      0   
+KvFWdNadmk4EsYLM1YJpMmfpZ6dWW+F6Gch4+AyI7I=       NaN  2.000000      0   
xar5CEm9tD/GwgRa9sjN0Fe/SeH7njj5XQHjBi+MRPE=       NaN  0.000000      0   
ZDBfOAFk7mSNPn5Ui39z9U3NY87u2kCbhmlPRZzJPBw=       NaN  0.000000      0   

                                                                 num_75  ...  \
                                                   std      mean    sum  ...   
msno                                                                     ...   
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  0.987096  0.846154      6  ...   
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  2.747960  1.525641     82  ...   
PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=  0.857493  0.500000     10  ...   
KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=  1.129319  1.166667     14  ...   
oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=  1.987247  1.500000     83  ...   
...                                                ...       ...    ...  ...   
yyPt5tJrg0A+u3ELliQHaxWCKHKuxHTJkBHkLvaGRuk=       NaN  0.000000      1  ...   
5EdEfCx2H/HCMLiFDgTyViSzMPLiPJrA6EFW8mqbGn8=       NaN  0.000000      0  ...   
+KvFWdNadmk4EsYLM1YJpMmfpZ6dWW+F6Gch4+AyI7I=       NaN  0.000000      0  ...   
xar5CEm9tD/GwgRa9sjN0Fe/SeH7njj5XQHjBi+MRPE=       NaN  0.000000      0  ...   
ZDBfOAFk7mSNPn5Ui39z9U3NY87u2kCbhmlPRZzJPBw=       NaN  0.000000      0  ...   

                                               num_985 num_100             \
                                                  mean     sum        std   
msno                                                                        
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  0.538462     106   9.118170   
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  0.794872    3694  62.032454   
PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=  0.277778     267  17.527290   
KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=  0.541667     263  10.860696   
oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=  0.966667    2984  51.771292   
...                                                ...     ...        ...   
yyPt5tJrg0A+u3ELliQHaxWCKHKuxHTJkBHkLvaGRuk=  0.00

In [17]:
result.columns = ['_'.join(col).strip() for col in result.columns.values]
result.columns

Index(['date_min', 'date_max', 'num_25_sum', 'num_25_count', 'num_25_std',
       'num_25_mean', 'num_50_sum', 'num_50_std', 'num_50_mean', 'num_75_sum',
       'num_75_std', 'num_75_mean', 'num_985_sum', 'num_985_std',
       'num_985_mean', 'num_100_sum', 'num_100_std', 'num_100_mean',
       'num_unq_sum', 'num_unq_std', 'num_unq_mean', 'total_secs_sum',
       'total_secs_std', 'total_secs_mean'],
      dtype='object')

In [34]:
#### Taking 400M rows to process data 

In [4]:
import time
size=5e7  # Taking 50 million
user_log_chunks = pd.read_csv('user_logs.csv',chunksize=size,index_col=['msno'])
start_time = time.time()
for i,user_log_chunk in enumerate(user_log_chunks):
    print(user_log_chunk.shape)
    grouped_object=user_log_chunk.groupby(user_log_chunk.index,sort=False) # not sorting results in a minor speedup
    func = {'date':['min','max'], 
           'num_25':['sum','count','mean'],'num_50':['sum','mean'],
           'num_75':['sum','mean'],'num_985':['sum','mean'],
           'num_100':['sum','mean'],'num_unq':['sum','mean'],'total_secs':['sum','mean']}
    if(i==0):  
        final_result=grouped_object.agg(func)
        print(len(final_result))
        print("Round ",i,"took %s seconds" % (time.time() - start_time))
        
    else:
        final_result =final_result.append(grouped_object.agg(func))
        print(len(final_result))
        print("Round ",i,"took %s seconds" % (time.time() - start_time))
        

(50000000, 8)
2027722
Round  0 took 194.0709867477417 seconds
(50000000, 8)
4054258
Round  1 took 391.77291774749756 seconds
(50000000, 8)
6080198
Round  2 took 594.5194110870361 seconds
(50000000, 8)
8107488
Round  3 took 801.7233140468597 seconds
(50000000, 8)
10134310
Round  4 took 991.2351679801941 seconds
(50000000, 8)
12160338
Round  5 took 1190.949702501297 seconds
(50000000, 8)
14187533
Round  6 took 1392.7482976913452 seconds
(42106543, 8)
16133886
Round  7 took 1548.9313991069794 seconds


In [5]:
final_result 

date           num_25        \
                                                   min       max    sum count   
msno                                                                            
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  20150326  20150716     75    13   
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150105  20170111    315    78   
PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=  20151201  20170125     18    18   
KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=  20150803  20170201     89    24   
oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=  20150205  20160727    167    60   
...                                                ...       ...    ...   ...   
CzQH4kawK4fk8WLTVA4Nh08Hd5ce2l8Tgtu/hHFsCTA=  20170217  20170217      1     1   
KFzaAx7Nk7ROMQA/B4HssFllae/8a/5rqoqBdeYAfTI=  20170216  20170216      1     1   
BS1wRL5jF6AloFSgvOHap42KomIOWboakt32AxiAmto=  20170217  20170217    100     1   
rVVkO1e1yZkUlKy621M70v+gBaK0HCuPrUoea1OsxYo=  20170220  20170220      6     1   
AKjgT/J+RyvwdBV8iDKIRcgCZej1AQeJYUdSvpxy2mw=  20170227  20170227     32     1   

                                                         num_50             \
                                                    mean    sum       mean   
msno                                                                         
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=    5.769231     11   0.846154   
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=    4.038462    119   1.525641   
PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=    1.000000      9   0.500000   
KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=    3.708333     28   1.166667   
oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=    2.783333     90   1.500000   
...                                                  ...    ...        ...   
CzQH4kawK4fk8WLTVA4Nh08Hd5ce2l8Tgtu/hHFsCTA=    1.000000      0   0.000000   
KFzaAx7Nk7ROMQA/B4HssFllae/8a/5rqoqBdeYAfTI=    1.000000      0   0.000000   
BS1wRL5jF6AloFSgvOHap42KomIOWboakt32AxiAmto=  100.000000     11  11.000000   
rVVkO1e1yZkUlKy621M70v+gBaK0HCuPrUoea1OsxYo=    6.000000      2   2.000000   
AKjgT/J+RyvwdBV8iDKIRcgCZej1AQeJYUdSvpxy2mw=   32.000000      2   2.000000   

                                             num_75           num_985  \
                                                sum      mean     sum   
msno                                                                    
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=      6  0.461538       7   
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=     82  1.051282      62   
PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=     10  0.555556       5   
KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=     14  0.583333      13   
oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=     83  1.383333      58   
...                                             ...       ...     ...   
CzQH4kawK4fk8WLTVA4Nh08Hd5ce2l8Tgtu/hHFsCTA=      0  0.000000       0   
KFzaAx7Nk7ROMQA/B4HssFllae/8a/5rqoqBdeYAfTI=      0  0.000000       0   
BS1wRL5jF6AloFSgvOHap42KomIOWboakt32AxiAmto=      4  4.000000       2   
rVVkO1e1yZkUlKy621M70v+gBaK0HCuPrUoea1OsxYo=      4  4.000000       0   
AKjgT/J+RyvwdBV8iDKIRcgCZej1AQeJYUdSvpxy2mw=      0  0.000000       1   

                                                       num_100             \
                                                  mean     sum       mean   
msno                                                                        
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  0.538462     106   8.153846   
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  0.794872    3694  47.358974   
PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=  0.277778     267  14.833333   
KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=  0.541667     263  10.958333   
oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=  0.966667    2984  49.733333   
...                                                ...     ...        ...   
CzQH4kawK4fk8WLTVA4Nh08Hd5ce2l8Tgtu/hHFsCTA=  0.000000      10  10.000000   
KFzaAx7Nk7ROMQA/B4HssFllae/8a

In [6]:
final_result.columns = ['_'.join(col).strip() for col in final_result.columns.values]
final_result.columns

Index(['date_min', 'date_max', 'num_25_sum', 'num_25_count', 'num_25_mean',
       'num_50_sum', 'num_50_mean', 'num_75_sum', 'num_75_mean', 'num_985_sum',
       'num_985_mean', 'num_100_sum', 'num_100_mean', 'num_unq_sum',
       'num_unq_mean', 'total_secs_sum', 'total_secs_mean'],
      dtype='object')

In [7]:
final_result = final_result.reset_index()

In [8]:
final_result.columns

Index(['msno', 'date_min', 'date_max', 'num_25_sum', 'num_25_count',
       'num_25_mean', 'num_50_sum', 'num_50_mean', 'num_75_sum', 'num_75_mean',
       'num_985_sum', 'num_985_mean', 'num_100_sum', 'num_100_mean',
       'num_unq_sum', 'num_unq_mean', 'total_secs_sum', 'total_secs_mean'],
      dtype='object')

In [9]:
final_result.to_csv('all_data_user_logs.csv')

In [16]:
import pandas as pd 
final_user_logs = pd.read_csv('all_data_user_logs.csv')

In [17]:
final_user_logs = final_user_logs.drop('Unnamed: 0',axis=1)

In [18]:
final_user_logs.head()

,msno,date_min,date_max,num_25_sum,num_25_count,num_25_mean,num_50_sum,num_50_mean,num_75_sum,num_75_mean,num_985_sum,num_985_mean,num_100_sum,num_100_mean,num_unq_sum,num_unq_mean,total_secs_sum,total_secs_mean
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150326,20150716,75,13,5.769231,11,0.846154,6,0.461538,7,0.538462,106,8.153846,124,9.538462,30483.466,2344.882000
1,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150105,20170111,315,78,4.038462,119,1.525641,82,1.051282,62,0.794872,3694,47.358974,2925,37.500000,975278.009,12503.564218
2,PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=,20151201,20170125,18,18,1.000000,9,0.500000,10,0.555556,5,0.277778,267,14.833333,252,14.000000,66812.551,3711.808389
3,KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=,20150803,20170201,89,24,3.708333,28,1.166667,14,0.583333,13,0.541667,263,10.958333,354,14.750000,70205.608,2925.233667
4,oy2721XIrBu+MEh8XjPTlt5b7gCronoHf/oFN+YwIv8=,20150205,20160727,167,60,2.783333,90,1.500000,83,1.383333,58,0.966667,2984,49.733333,2133,35.550000,705093.332,11751.555533


In [19]:
train = pd.merge(train,final_user_logs,how='left',on='msno')
test = pd.merge(test,final_user_logs,how='left',on='msno')

In [22]:
final_user_logs= []

In [23]:
test

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,num_75_sum,num_75_mean,num_985_sum,num_985_mean,num_100_sum,num_100_mean,num_unq_sum,num_unq_mean,total_secs_sum,total_secs_mean
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,1.0,0.0,NaN,7.0,20150718.0,41,30,99,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,4.0,4.000000,159.516,159.516000
1,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,1.0,0.0,NaN,7.0,20150718.0,41,30,99,...,1.0,0.333333,0.0,0.000000,1.0,0.333333,8.0,2.666667,672.416,224.138667
2,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,4.0,28.0,1.0,9.0,20051030.0,34,30,149,...,242.0,2.688889,269.0,2.988889,3951.0,43.900000,5928.0,65.866667,1129730.170,12552.557444
3,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,4.0,28.0,1.0,9.0,20051030.0,34,30,149,...,349.0,3.323810,335.0,3.190476,5046.0,48.057143,7111.0,67.723810,1432564.218,13643.468743
4,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,4.0,28.0,1.0,9.0,20051030.0,34,30,149,...,257.0,2.763441,279.0,3.000000,4553.0,48.956989,6099.0,65.580645,1260934.278,13558.433097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595171,YhPGjMES6gPOZMm33BmxwM4hg5I1gbpBM5Be16NXZoI=,0,1.0,0.0,NaN,7.0,20130820.0,41,30,99,...,17.0,0.314815,32.0,0.592593,1006.0,18.629630,722.0,13.370370,247714.697,4587.309204
5595172,YhPGjMES6gPOZMm33BmxwM4hg5I1gbpBM5Be16NXZoI=,0,1.0,0.0,NaN,7.0,20130820.0,41,30,99,...,19.0,0.306452,23.0,0.370968,919.0,14.822581,668.0,10.774194,228004.534,3677.492484
5595173,YhPGjMES6gPOZMm33BmxwM4hg5I1gbpBM5Be16NXZoI=,0,1.0,0.0,NaN,7.0,20130820.0,41,30,99,...,21.0,0.437500,15.0,0.312500,671.0,13.979167,492.0,10.250000,168671.750,3513.994792
5595174,YhPGjMES6gPOZMm33BmxwM4hg5I1gbpBM5Be16NXZoI=,0,1.0,0.0,NaN,7.0,20130820.0,41,30,99,...,21.0,0.375000,28.0,0.500000,776.0,13.857143,560.0,10.000000,198469.870,3544.104821


In [24]:
train

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,num_75_sum,num_75_mean,num_985_sum,num_985_mean,num_100_sum,num_100_mean,num_unq_sum,num_unq_mean,total_secs_sum,total_secs_mean
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,2.0,9.0,20050406.0,38,30,149,...,6.0,0.857143,2.0,0.285714,133.0,19.000000,154.0,22.000000,35323.665,5046.237857
1,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,2.0,9.0,20050406.0,38,30,149,...,1.0,0.250000,1.0,0.250000,19.0,4.750000,19.0,4.750000,5595.883,1398.970750
2,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,2.0,9.0,20050406.0,38,30,149,...,1.0,0.333333,1.0,0.333333,33.0,11.000000,42.0,14.000000,8137.030,2712.343333
3,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,2.0,9.0,20050406.0,38,30,149,...,6.0,1.000000,0.0,0.000000,88.0,14.666667,100.0,16.666667,22585.123,3764.187167
4,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,2.0,9.0,20050406.0,38,30,149,...,5.0,0.833333,7.0,1.166667,136.0,22.666667,147.0,24.500000,36410.153,6068.358833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12727771,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,5.0,25.0,2.0,9.0,20110905.0,39,30,149,...,27.0,0.450000,36.0,0.600000,804.0,13.400000,900.0,15.000000,198848.377,3314.139617
12727772,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,5.0,25.0,2.0,9.0,20110905.0,39,30,149,...,24.0,0.406780,57.0,0.966102,839.0,14.220339,926.0,15.694915,212467.638,3601.146407
12727773,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,5.0,25.0,2.0,9.0,20110905.0,39,30,149,...,42.0,0.646154,60.0,0.923077,1212.0,18.646154,1279.0,19.676923,305306.026,4697.015785
12727774,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,5.0,25.0,2.0,9.0,20110905.0,39,30,149,...,39.0,0.780000,48.0,0.960000,713.0,14.260000,900.0,18.000000,186518.182,3730.363640


In [25]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
#train = train.groupby('msno')

In [ ]:
#train = train.first().reset_index()

In [ ]:
#train 

In [26]:
test = test.groupby('msno')

In [27]:
test = test.first().reset_index()

In [28]:
test

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,num_75_sum,num_75_mean,num_985_sum,num_985_mean,num_100_sum,num_100_mean,num_unq_sum,num_unq_mean,total_secs_sum,total_secs_mean
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,1.0,0.0,0.0,7.0,20140608.0,41,30,99,...,30.0,2.500000,80.0,6.666667,264.0,22.000000,307.0,25.583333,89992.430,7499.369167
1,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0,1.0,0.0,0.0,7.0,20140927.0,41,30,149,...,202.0,2.404762,199.0,2.369048,3464.0,41.238095,3623.0,43.130952,976483.723,11624.806226
2,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0,18.0,21.0,1.0,7.0,20160315.0,41,30,149,...,62.0,1.476190,73.0,1.738095,653.0,15.547619,534.0,12.714286,200355.988,4770.380667
3,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,0,1.0,0.0,0.0,7.0,20160320.0,41,30,149,...,24.0,1.411765,25.0,1.470588,134.0,7.882353,279.0,16.411765,43810.515,2577.089118
4,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,0,1.0,0.0,0.0,9.0,20150807.0,38,30,149,...,18.0,0.642857,15.0,0.535714,970.0,34.642857,1013.0,36.178571,251368.052,8977.430429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907466,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,0,4.0,24.0,1.0,7.0,20120306.0,41,30,99,...,72.0,1.200000,82.0,1.366667,3799.0,63.316667,3030.0,50.500000,1004013.303,16733.555050
907467,zzxi7n5xoTYo9Q3VTygLWvl/rBDcexwaeAry0yK7Q0E=,0,0.0,0.0,0.0,0.0,0.0,41,30,99,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.000000
907468,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,0,5.0,31.0,1.0,3.0,20150430.0,36,30,180,...,62.0,3.444444,61.0,3.388889,731.0,40.611111,752.0,41.777778,197050.868,10947.270444
907469,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,0,9.0,0.0,0.0,7.0,20130515.0,29,30,180,...,50.0,0.793651,76.0,1.206349,3474.0,55.142857,2958.0,46.952381,804498.088,12769.810921


In [34]:
train.to_csv('final_train.csv')
test.to_csv('final_test.csv')

In [29]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

In [30]:
cols = [c for c in train.columns if c not in ['is_churn','msno']]

In [31]:
%%time
def xgb_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'log_loss',log_loss(labels, preds)

fold = 1
for i in range(fold):
    params = {
        'eta': 0.02, 
        'max_depth': 7,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'seed': i,
        'silent': True
    }
    x1, x2, y1, y2 = train_test_split(train[cols], train['is_churn'], test_size=0.3, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 150,  watchlist, feval=xgb_score, maximize=False, verbose_eval=50, early_stopping_rounds=15000) 
    if i != 0:
        pred += model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
    else:
        pred = model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
pred /= fold
test['is_churn'] = pred.clip(0.0000001, 0.999999)
test[['msno','is_churn']].to_csv('submission3.csv', index=False)

[22:00:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:00:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\gbm\gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	train-logloss:0.68296	valid-logloss:0.67669	train-log_loss:1.10232	valid-log_loss:1.10075
Multiple eval metrics have been passed: 'valid-log_loss' will be used for early stopping.

Will train until valid-log_loss hasn't improved in 15000 rounds.
[50]	train-logloss:0.25068	valid-logloss:0.24957	train-log_loss:nan	valid-log_loss:nan
[100]	train-

In [32]:
a = pd.read_csv('submission3.csv')

In [33]:
a

,msno,is_churn
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.279326
1,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0.279326
2,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0.279326
3,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,0.279326
4,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,0.303257
...,...,...
907466,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,0.279326
907467,zzxi7n5xoTYo9Q3VTygLWvl/rBDcexwaeAry0yK7Q0E=,0.279326
907468,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,0.380806
907469,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,0.317393
